A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
test

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
set_seed(3407)

In [2]:
import pickle

class SortDataset(Dataset):
    """ 
    Dataset for the Sort problem. E.g. for problem length 6:
    Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
    Which will feed into the transformer concatenated as:
    input:  0 0 2 1 0 1 0 0 0 1 1
    output: I I I I I 0 0 0 1 1 2
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=6, num_digits=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
        self.num_digits = num_digits
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length * 2 - 1

    def __getitem__(self, idx):
        
        # use rejection sampling to generate an input example from the desired split
        while True:
            # generate some random integers
            inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
            # half of the time let's try to boost the number of examples that 
            # have a large number of repeats, as this is what the model seems to struggle
            # with later in training, and they are kind of rate
            if torch.rand(1).item() < 0.5:
                if inp.unique().nelement() > self.length // 2:
                    # too many unqiue digits, re-sample
                    continue
            # figure out if this generated example is train or test based on its hash
            h = hash(pickle.dumps(inp.tolist()))
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        # solve the task: i.e. sort
        sol = torch.sort(inp)[0] 	# sort returns a tuple of two tensors, the sorted tensor and the position of the elements in the original; 

        # concatenate the problem specification and the solution
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:self.length-1] = -1
        return x, y


In [3]:
# print an example instance of the dataset
train_dataset = SortDataset('train')
test_dataset = SortDataset('test')
x, y = train_dataset[0]
for a, b in zip(x,y):
    print(int(a),int(b))

2 -1
0 -1
0 -1
1 -1
1 -1
1 0
0 0
0 1
1 1
1 1
1 2


In [ ]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.09M


In [5]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 2000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

running on device cuda


In [6]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 1.07207
iter_dt 13.92ms; iter 100: train loss 0.11113
iter_dt 15.82ms; iter 200: train loss 0.06465
iter_dt 18.19ms; iter 300: train loss 0.05093
iter_dt 13.31ms; iter 400: train loss 0.02231
iter_dt 12.76ms; iter 500: train loss 0.00663
iter_dt 19.54ms; iter 600: train loss 0.03124
iter_dt 12.52ms; iter 700: train loss 0.01977
iter_dt 12.06ms; iter 800: train loss 0.05053
iter_dt 13.84ms; iter 900: train loss 0.00747
iter_dt 12.04ms; iter 1000: train loss 0.00475
iter_dt 14.23ms; iter 1100: train loss 0.03746
iter_dt 12.46ms; iter 1200: train loss 0.04496
iter_dt 12.92ms; iter 1300: train loss 0.00192
iter_dt 12.56ms; iter 1400: train loss 0.00070
iter_dt 12.57ms; iter 1500: train loss 0.01474
iter_dt 12.40ms; iter 1600: train loss 0.00312
iter_dt 12.85ms; iter 1700: train loss 0.00503
iter_dt 12.43ms; iter 1800: train loss 0.00080
iter_dt 23.70ms; iter 1900: train loss 0.01967


In [7]:
# now let's perform some evaluation
model.eval();

In [8]:
def eval_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        # isolate the input pattern alone
        inp = x[:, :n]
        sol = y[:, -n:]
        # let the model sample the rest of the sequence
        cat = model.generate(inp, n, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, n:] # isolate the filled in sequence
        # compare the predicted sequence to the true sequence
        correct = (sol == sol_candidate).all(1).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line haha
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            if not correct[i] and mistakes_printed_already < 3: # only print up to 5 mistakes to get a sense
                mistakes_printed_already += 1
                print("GPT claims that %s sorted is %s but gt is %s" % (inp[i].tolist(), sol_candidate[i].tolist(), sol[i].tolist()))
        if max_batches is not None and b+1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():	#unnecessary as generate already has no_grad decorator;
    train_score = eval_split(trainer, 'train', max_batches=50)
    test_score  = eval_split(trainer, 'test',  max_batches=50)

train final score: 5000/5000 = 100.00% correct
test final score: 5000/5000 = 100.00% correct


In [26]:
# let's run a random given sequence through the model as well
n = train_dataset.length # naugy direct access shrug
inp = torch.tensor([[0, 0, 2, 1, 0, 1]], dtype=torch.long).to(trainer.device)
assert inp[0].nelement() == n
with torch.no_grad():
    cat = model.generate(inp, n, do_sample=False)
sol = torch.sort(inp[0])[0]
sol_candidate = cat[:, n:]
print('input sequence  :', inp.tolist())
print('predicted sorted:', sol_candidate.tolist())
print('gt sort         :', sol.tolist())
print('matches         :', bool((sol == sol_candidate).all()))

input sequence  : [[0, 0, 2, 1, 0, 1]]
predicted sorted: [[0, 0, 0, 1, 1, 2]]
gt sort         : [0, 0, 0, 1, 1, 2]
matches         : True


In [9]:
model.state_dict()

OrderedDict([('transformer.wte.weight',
              tensor([[-1.7782e-02, -1.0397e-02,  4.3755e-03, -9.4283e-03, -1.5903e-02,
                        2.2505e-05, -6.1369e-02,  2.6203e-02,  4.7436e-02,  3.7454e-03,
                       -4.1899e-02,  2.6155e-02,  2.2934e-03, -2.4311e-02, -2.1005e-02,
                        2.6950e-02,  8.4053e-03,  3.0998e-02,  1.2100e-02, -1.4270e-02,
                       -8.1398e-03,  1.7157e-02,  2.5038e-02,  1.5144e-02,  7.6834e-02,
                       -3.5189e-02,  1.1483e-03,  6.1134e-04,  1.2080e-02,  1.4078e-02,
                        6.2777e-02, -6.0420e-04,  2.4666e-02, -2.3203e-02, -1.3324e-02,
                       -4.9718e-02, -2.1225e-02,  4.8994e-02,  2.0825e-02,  1.5686e-02,
                        9.4109e-03, -8.2799e-03,  3.2943e-02, -2.9890e-02,  2.6659e-02,
                        1.1032e-02, -7.0563e-03, -3.6639e-02],
                      [ 1.1626e-03,  6.7442e-02,  1.2697e-02, -9.5651e-03, -5.7280e-04,
                 